In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load your data from the Excel file
finance_records = pd.read_excel('2024-03-23.xlsx')

# Filter out transfer transactions and the "Modified Bal." category
finance_records = finance_records[~finance_records['Income/Expense'].str.contains('Transfer')]
finance_records = finance_records[finance_records['Category'] != 'Modified Bal.']

# Extract the year and month from the 'Period' column
finance_records['Year'] = finance_records['Period'].dt.year
finance_records['Month'] = finance_records['Period'].dt.month

# Separate income and expenses
income_records = finance_records[finance_records['Income/Expense'] == 'Income']
expense_records = finance_records[finance_records['Income/Expense'] == 'Exp.']

# Calculate monthly totals
monthly_expense_totals = expense_records.groupby(['Year', 'Month'])['Amount'].sum().reset_index()
monthly_income_totals = income_records.groupby(['Year', 'Month'])['Amount'].sum().reset_index()

# Specify the year you want to focus on (e.g., 2023)
focus_year = 2024

# Plot Expense Distribution by Category (for the specified year)
recent_year_expenses = expense_records[expense_records['Year'] == focus_year].groupby('Category')['Amount'].sum().reset_index()
fig = px.pie(recent_year_expenses, values='Amount', names='Category', title=f'Expense Distribution by Category ({focus_year})')
fig.show()

# Plot Income Distribution by Category (for the specified year)
recent_year_income = income_records[income_records['Year'] == focus_year].groupby('Category')['Amount'].sum().reset_index()
fig = px.pie(recent_year_income, values='Amount', names='Category', title=f'Income Distribution by Category ({focus_year})')
fig.show()

# Plot Monthly Expense Trends Over Time (for the entire period)
fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_expense_totals.index, y=monthly_expense_totals['Amount'], mode='lines+markers', name='Expenses', line=dict(color='red')))
fig.update_layout(title='Monthly Expense Trends Over Time', xaxis_title='Month', yaxis_title='Amount (MYR)')
fig.update_xaxes(tickvals=monthly_expense_totals.index, ticktext=[f'{year}-{month:02}' for year, month in zip(monthly_expense_totals['Year'], monthly_expense_totals['Month'])], tickangle=45)
fig.show()

# Plot Monthly Income Trends Over Time (for the entire period)
fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_income_totals.index, y=monthly_income_totals['Amount'], mode='lines+markers', name='Income', line=dict(color='green')))
fig.update_layout(title='Monthly Income Trends Over Time', xaxis_title='Month', yaxis_title='Amount (MYR)')
fig.update_xaxes(tickvals=monthly_income_totals.index, ticktext=[f'{year}-{month:02}' for year, month in zip(monthly_income_totals['Year'], monthly_income_totals['Month'])], tickangle=45)
fig.show()

# Calculate Expense vs. Income Ratio Over Time (for the entire period)
expense_income_ratio = monthly_expense_totals['Amount'] / monthly_income_totals['Amount']
fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_expense_totals.index, y=expense_income_ratio, mode='lines+markers', name='Expense/Income Ratio', line=dict(color='orange')))
fig.update_layout(title='Expense vs. Income Ratio Over Time', xaxis_title='Month', yaxis_title='Ratio')
fig.update_xaxes(tickvals=monthly_expense_totals.index, ticktext=[f'{year}-{month:02}' for year, month in zip(monthly_expense_totals['Year'], monthly_expense_totals['Month'])], tickangle=45)
fig.show()

# Seasonal Spending Patterns (by month)
monthly_expense_means = monthly_expense_totals.groupby('Month')['Amount'].mean().reset_index()
fig = px.bar(monthly_expense_means, x='Month', y='Amount', title='Seasonal Spending Patterns (Average Monthly Expenses)', text='Amount', color_discrete_sequence=['purple'])
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_xaxes(tickvals=monthly_expense_means['Month'], ticktext=[f'Month {month}' for month in monthly_expense_means['Month']])
fig.show()

# Budget Variance Analysis (assuming you have a budget)
monthly_budget = 3500  # Replace with your actual budget for each month
monthly_actual_expenses = monthly_expense_totals.groupby('Month')['Amount'].sum().reset_index()
budget_vs_actual = monthly_budget - monthly_actual_expenses['Amount']
fig = px.bar(monthly_actual_expenses, x='Month', y='Amount', title='Monthly Budget vs. Actual Spending', text='Amount')
fig.add_trace(go.Scatter(x=monthly_actual_expenses['Month'], y=budget_vs_actual, mode='lines', name='Budget Variance', line=dict(color='red')))
fig.update_xaxes(tickvals=monthly_actual_expenses['Month'], ticktext=[f'Month {month}' for month in monthly_actual_expenses['Month']])
fig.update_layout(yaxis_title='Amount (MYR)')
fig.show()

# Top Expense Subcategories (for the specified year)
recent_year_expense_subcategories = expense_records[expense_records['Year'] == focus_year].groupby('Subcategory')['Amount'].sum().nlargest(3).reset_index()
fig = px.bar(recent_year_expense_subcategories, x='Subcategory', y='Amount', title=f'Top 3 Expense Subcategories ({focus_year})', text='Amount')
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

# Top Merchants or Accounts
top_merchants = finance_records[(finance_records['Income/Expense'] == 'Exp.') & (finance_records['Year'] == focus_year)].groupby('Accounts')['Amount'].sum().nlargest(3).reset_index()
fig = px.bar(top_merchants, x='Accounts', y='Amount', title=f'Top 3 Expense Merchants/Accounts ({focus_year})', text='Amount')
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

# 1. Expense Distribution by Subcategory (for the specified year)
recent_year_expense_subcategories = expense_records[expense_records['Year'] == focus_year].groupby('Subcategory')['Amount'].sum().reset_index()

# Keep the top 15 subcategories and group the rest as 'Others'
top_15_subcategories = recent_year_expense_subcategories.nlargest(22, 'Amount')
other_subcategories = recent_year_expense_subcategories[~recent_year_expense_subcategories['Subcategory'].isin(top_15_subcategories['Subcategory'])]
other_total_amount = other_subcategories['Amount'].sum()

# Add 'Others' as a new row using pd.concat
top_15_subcategories = pd.concat([top_15_subcategories, pd.DataFrame({'Subcategory': ['Others'], 'Amount': [other_total_amount]})], ignore_index=True)

fig = px.pie(top_15_subcategories, values='Amount', names='Subcategory', title=f'Expense Distribution by Subcategory ({focus_year})')

fig.show()


# 2. Top Income Sources (for the specified year)
top_income_sources = income_records[income_records['Year'] == focus_year].groupby('Accounts')['Amount'].sum().nlargest(3).reset_index()
fig = px.bar(top_income_sources, x='Accounts', y='Amount', title=f'Top 3 Income Sources ({focus_year})', text='Amount')
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

# 3. Expense Trends by Category Over Time (for all years)
expense_category_trends = expense_records.groupby(['Year', 'Category'])['Amount'].sum().reset_index()
fig = px.line(expense_category_trends, x='Year', y='Amount', color='Category', title='Expense Trends by Category Over Time')
fig.show()

# 4. Income Trends by Category Over Time (for all years)
income_category_trends = income_records.groupby(['Year', 'Category'])['Amount'].sum().reset_index()
fig = px.line(income_category_trends, x='Year', y='Amount', color='Category', title='Income Trends by Category Over Time')
fig.show()

# 5. Expense vs. Income Over Time (for all years)
total_expense_vs_income = pd.merge(monthly_expense_totals, monthly_income_totals, on=['Year', 'Month'], suffixes=('_Expense', '_Income'))
fig = px.line(total_expense_vs_income, x=['{0}-{1:02}'.format(year, month) for year, month in zip(total_expense_vs_income['Year'], total_expense_vs_income['Month'])], y=['Amount_Expense', 'Amount_Income'], title='Expense vs. Income Over Time')
fig.update_traces(mode='lines+markers')
fig.update_xaxes(tickvals=total_expense_vs_income.index, ticktext=[f'{year}-{month:02}' for year, month in zip(total_expense_vs_income['Year'], total_expense_vs_income['Month'])], tickangle=45)
fig.show()

# 6. Savings Over Time (for all years)
total_savings = monthly_income_totals['Amount'] - monthly_expense_totals['Amount']
fig = px.line(total_savings, x=['{0}-{1:02}'.format(year, month) for year, month in zip(monthly_expense_totals['Year'], monthly_expense_totals['Month'])], y=total_savings, title='Savings Over Time')
fig.update_traces(mode='lines+markers')
fig.update_xaxes(tickvals=total_savings.index, ticktext=[f'{year}-{month:02}' for year, month in zip(monthly_expense_totals['Year'], monthly_expense_totals['Month'])], tickangle=45)
fig.show()

# Calculate Expense vs. Income Ratio (for the specified year)
recent_year_total_income = monthly_income_totals[(monthly_income_totals['Year'] == focus_year)]['Amount'].sum()
recent_year_total_expenses = monthly_expense_totals[(monthly_expense_totals['Year'] == focus_year)]['Amount'].sum()
expense_income_ratio = recent_year_total_expenses / recent_year_total_income
print(f'\nExpense vs. Income Ratio ({focus_year}): {expense_income_ratio:.2f}')

# Calculate Savings Rate (for the specified year)
savings_rate = (recent_year_total_income - recent_year_total_expenses) / recent_year_total_income
print(f'\nSavings Rate ({focus_year}): {savings_rate:.2%}')


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed